# TMT (Total Miscoordination Time) Analysis for All Scenarios

This notebook provides a comprehensive analysis of relay coordination data and calculates TMT by operational scenario. Designed for international research collaboration and easy sharing.


In [20]:
# %% Import Libraries
import json
import os
import copy
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import re
from pathlib import Path
from collections import defaultdict
import traceback
from scipy import stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# %% Configuration
# Set default Plotly template
pio.templates.default = "plotly_white"

# --- Portable Configuration for International Researchers ---
class ProjectConfig:
    """Portable configuration for easy sharing and collaboration with international researchers"""
    
    def __init__(self, project_name="RelayCoordination", data_file_name="automation_results.json"):
        # Auto-detect project structure - works on any system
        current_dir = Path.cwd()
        if current_dir.name == "notebooks":
            self.PROJECT_ROOT = current_dir.parent
        elif current_dir.name == "analysis":
            self.PROJECT_ROOT = current_dir.parent
        else:
            self.PROJECT_ROOT = current_dir
        
        # Ensure we're pointing to the correct project root
        # Look for the data directory to confirm we're in the right place
        if not (self.PROJECT_ROOT / "data" / "raw").exists():
            # If we're in analysis/notebooks, go up two levels
            if current_dir.name == "notebooks" and current_dir.parent.name == "analysis":
                self.PROJECT_ROOT = current_dir.parent.parent
            # If we're in analysis, go up one level
            elif current_dir.name == "analysis":
                self.PROJECT_ROOT = current_dir.parent
        
        # Directory configuration - PORTABLE AND REPLICABLE
        self.DATA_DIR = self.PROJECT_ROOT / "data"
        self.RAW_DATA_DIR = self.DATA_DIR / "raw"
        self.PROCESSED_DATA_DIR = self.DATA_DIR / "processed"
        self.RESULTS_DIR = self.PROJECT_ROOT / "results"
        
        # Input and output files - WITH TIMESTAMP TO AVOID OVERWRITING
        self.INPUT_FILE = self.RAW_DATA_DIR / data_file_name
        self.OUTPUT_DIR = self.RESULTS_DIR / "plots" / "tmt_analysis"
        self.TABLES_DIR = self.RESULTS_DIR / "tables"
        
        # Create necessary directories
        self.OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
        self.TABLES_DIR.mkdir(parents=True, exist_ok=True)
        
        # Analysis configuration - ADJUSTABLE BY RESEARCHER
        self.CTI = 0.2  # Typical coordination time interval (seconds)
        self.MIN_COORDINATION_THRESHOLD = 80.0  # Minimum acceptable coordination threshold (%)
        self.CRITICAL_TMT_THRESHOLD = -20.0  # Critical TMT threshold
        
        # Graphics configuration - OPTIMIZED FOR PUBLICATIONS
        self.FIG_WIDTH = 1200
        self.FIG_HEIGHT = 600
        self.TITLE_FONT_SIZE = 20
        self.AXIS_LABEL_FONT_SIZE = 16
        self.TICK_FONT_SIZE = 12
        self.LEGEND_TITLE_FONT_SIZE = 14
        self.LEGEND_FONT_SIZE = 12
        self.TEXT_ON_BAR_FONT_SIZE = 10
        self.PLOT_TEMPLATE = "plotly_white"
        
        # Colors for graphics - PROFESSIONAL AND CONSISTENT
        self.COLORS = {
            'coordinated': '#2E8B57',      # Sea green
            'uncoordinated': '#DC143C',    # Crimson
            'critical': '#FF4500',         # Orange red
            'warning': '#FFD700',          # Gold
            'good': '#32CD32',             # Lime green
            'primary': '#4169E1',          # Royal blue
            'secondary': '#9370DB'         # Medium purple
        }
        
        # Only the most important visualizations
        self.ESSENTIAL_VISUALIZATIONS = [
            'coordination_quality',
            'tmt_severity', 
            'metrics_dashboard'
        ]
    
    def print_config(self):
        """Print current project configuration"""
        print("="*60)
        print(f"PROJECT CONFIGURATION: {self.PROJECT_ROOT.name}")
        print("="*60)
        print(f"📁 Project root: {self.PROJECT_ROOT}")
        print(f"📊 Data file: {self.INPUT_FILE}")
        print(f"📈 Output directory: {self.OUTPUT_DIR}")
        print(f"📋 Tables directory: {self.TABLES_DIR}")
        print(f"✅ File exists: {self.INPUT_FILE.exists()}")
        print(f"⏱️  CTI configured: {self.CTI} seconds")
        print(f"🎯 Coordination threshold: {self.MIN_COORDINATION_THRESHOLD}%")
        print(f"⚠️  Critical TMT threshold: {self.CRITICAL_TMT_THRESHOLD}")
        print("="*60)
    
    def get_relative_paths(self):
        """Get relative paths for easy sharing"""
        return {
            'data_file': str(self.INPUT_FILE.relative_to(self.PROJECT_ROOT)),
            'output_dir': str(self.OUTPUT_DIR.relative_to(self.PROJECT_ROOT)),
            'tables_dir': str(self.TABLES_DIR.relative_to(self.PROJECT_ROOT))
        }

# Initialize configuration
config = ProjectConfig()
config.print_config()

# Show portable paths for easy sharing
print("\n🌍 PORTABLE PATHS FOR INTERNATIONAL SHARING:")
print("="*60)
portable_paths = config.get_relative_paths()
print("📁 Project Structure (relative paths):")
print("   • data_dir: data/")
print("   • raw_data_dir: data/raw/")
print("   • processed_data_dir: data/processed/")
print("   • results_dir: results/")
print("   • plots_dir: results/plots/tmt_analysis/")
print("   • tables_dir: results/tables/")

print("\n📊 Required Files:")
print(f"   • input_data: {portable_paths['data_file']}")

print("\n📈 Output Files (with timestamps):")
print("   • csv_results: results/tables/tmt_analysis_results_[timestamp].csv")
print("   • metrics_json: results/tables/metrics_summary_[timestamp].json")
print("   • report_txt: results/analysis_report_[timestamp].txt")
print("   • plots_html: results/plots/tmt_analysis/[plot_name]_[timestamp].html")
print("   • plots_png: results/plots/tmt_analysis/[plot_name]_[timestamp].png")

print("\n✅ This notebook is ready for international collaboration!")
print("="*60)


PROJECT CONFIGURATION: AutoDOC-MG
📁 Project root: /Users/gustavo/Documents/Projects/TESIS_UNAL/AutoDOC-MG
📊 Data file: /Users/gustavo/Documents/Projects/TESIS_UNAL/AutoDOC-MG/data/raw/automation_results.json
📈 Output directory: /Users/gustavo/Documents/Projects/TESIS_UNAL/AutoDOC-MG/results/plots/tmt_analysis
📋 Tables directory: /Users/gustavo/Documents/Projects/TESIS_UNAL/AutoDOC-MG/results/tables
✅ File exists: True
⏱️  CTI configured: 0.2 seconds
🎯 Coordination threshold: 80.0%
⚠️  Critical TMT threshold: -20.0

🌍 PORTABLE PATHS FOR INTERNATIONAL SHARING:
📁 Project Structure (relative paths):
   • data_dir: data/
   • raw_data_dir: data/raw/
   • processed_data_dir: data/processed/
   • results_dir: results/
   • plots_dir: results/plots/tmt_analysis/
   • tables_dir: results/tables/

📊 Required Files:
   • input_data: data/raw/automation_results.json

📈 Output Files (with timestamps):
   • csv_results: results/tables/tmt_analysis_results_[timestamp].csv
   • metrics_json: results/t

In [21]:
# %% Constantes de Estilo para Gráficos
FIG_WIDTH = 1200      # Ancho de la figura en píxeles
FIG_HEIGHT = 600      # Alto de la figura en píxeles
TITLE_FONT_SIZE = 20
AXIS_LABEL_FONT_SIZE = 16
TICK_FONT_SIZE = 12
LEGEND_TITLE_FONT_SIZE = 14
LEGEND_FONT_SIZE = 12
TEXT_ON_BAR_FONT_SIZE = 10
PLOT_TEMPLATE = "plotly_white"

print("Configuración de gráficos cargada.")


Configuración de gráficos cargada.


In [22]:
# %% Advanced Helper Functions
def extract_scenario_number(scenario_name):
    """
    Extract the first number found in the scenario name.
    If no number is found, returns a very large value to sort it at the end.
    """
    if not isinstance(scenario_name, str):
        return float('inf')
    match = re.search(r'\d+', scenario_name)
    if match:
        return int(match.group(0))
    else:
        print(f"Warning: No number found in scenario name '{scenario_name}'. Will be sorted at the end.")
        return float('inf')

def validate_file_path(file_path):
    """
    Validate that the file exists and is accessible.
    """
    if not file_path.exists():
        raise FileNotFoundError(f"File not found at path: {file_path}")
    
    if not file_path.is_file():
        raise ValueError(f"Path does not point to a file: {file_path}")
    
    return True

def load_relay_data(file_path):
    """
    Load and validate relay data from JSON file.
    """
    validate_file_path(file_path)
    
    print(f"📊 Loading data from: {file_path}")
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            relay_pairs_data = json.load(f)
    except json.JSONDecodeError as e:
        raise json.JSONDecodeError(f"Error decoding JSON: {e}")
    
    if not isinstance(relay_pairs_data, list):
        raise TypeError(f"Error: File {file_path} does not contain a main JSON list.")
    
    print(f"✅ Data loaded successfully. Total entries: {len(relay_pairs_data):,}")
    return relay_pairs_data

def create_portable_paths(config):
    """
    Create portable paths that work on any system for easy sharing.
    """
    return {
        'project_structure': {
            'data_dir': 'data/',
            'raw_data_dir': 'data/raw/',
            'processed_data_dir': 'data/processed/',
            'results_dir': 'results/',
            'plots_dir': 'results/plots/tmt_analysis/',
            'tables_dir': 'results/tables/'
        },
        'required_files': {
            'input_data': 'data/raw/automation_results.json'
        },
        'output_files': {
            'csv_results': 'results/tables/tmt_analysis_results_[timestamp].csv',
            'metrics_json': 'results/tables/metrics_summary_[timestamp].json',
            'report_txt': 'results/analysis_report_[timestamp].txt',
            'plots_html': 'results/plots/tmt_analysis/[plot_name]_[timestamp].html',
            'plots_png': 'results/plots/tmt_analysis/[plot_name]_[timestamp].png'
        }
    }

def calculate_advanced_metrics(df):
    """
    Calculate advanced metrics for coordination analysis.
    """
    metrics = {}
    
    # Basic metrics
    metrics['total_scenarios'] = len(df)
    metrics['total_pairs'] = df['Total Pares Válidos'].sum()
    metrics['coordinated_pairs'] = df['Pares Coordinados'].sum()
    metrics['uncoordinated_pairs'] = df['Pares Descoordinados'].sum()
    metrics['total_tmt'] = df['TMT'].sum()
    metrics['avg_coordination'] = df['Porcentaje Coordinación'].mean()
    
    # Statistical metrics
    metrics['coordination_std'] = df['Porcentaje Coordinación'].std()
    metrics['tmt_std'] = df['TMT'].std()
    metrics['coordination_median'] = df['Porcentaje Coordinación'].median()
    metrics['tmt_median'] = df['TMT'].median()
    
    # Quality metrics
    good_scenarios = df[df['Porcentaje Coordinación'] >= config.MIN_COORDINATION_THRESHOLD]
    metrics['good_scenarios_count'] = len(good_scenarios)
    metrics['good_scenarios_percentage'] = (len(good_scenarios) / len(df)) * 100
    
    critical_scenarios = df[df['TMT'] <= config.CRITICAL_TMT_THRESHOLD]
    metrics['critical_scenarios_count'] = len(critical_scenarios)
    metrics['critical_scenarios_percentage'] = (len(critical_scenarios) / len(df)) * 100
    
    # Distribution metrics
    metrics['coordination_range'] = df['Porcentaje Coordinación'].max() - df['Porcentaje Coordinación'].min()
    metrics['tmt_range'] = df['TMT'].max() - df['TMT'].min()
    
    return metrics

def generate_quality_assessment(metrics):
    """
    Generate a quality assessment based on calculated metrics.
    """
    assessment = {
        'overall_quality': 'CRITICAL',
        'coordination_status': 'INSUFFICIENT',
        'tmt_status': 'CRITICAL',
        'recommendations': []
    }
    
    # Coordination evaluation
    if metrics['avg_coordination'] >= 90:
        assessment['coordination_status'] = 'EXCELLENT'
    elif metrics['avg_coordination'] >= 80:
        assessment['coordination_status'] = 'GOOD'
    elif metrics['avg_coordination'] >= 60:
        assessment['coordination_status'] = 'REGULAR'
    else:
        assessment['coordination_status'] = 'INSUFFICIENT'
    
    # TMT evaluation
    if metrics['total_tmt'] >= -100:
        assessment['tmt_status'] = 'GOOD'
    elif metrics['total_tmt'] >= -500:
        assessment['tmt_status'] = 'REGULAR'
    else:
        assessment['tmt_status'] = 'CRITICAL'
    
    # Overall evaluation
    if assessment['coordination_status'] in ['EXCELLENT', 'GOOD'] and assessment['tmt_status'] in ['GOOD', 'REGULAR']:
        assessment['overall_quality'] = 'GOOD'
    elif assessment['coordination_status'] in ['REGULAR'] or assessment['tmt_status'] in ['REGULAR']:
        assessment['overall_quality'] = 'REGULAR'
    else:
        assessment['overall_quality'] = 'CRITICAL'
    
    # Generate recommendations
    if metrics['avg_coordination'] < 80:
        assessment['recommendations'].append("🔴 CRITICAL: Very low average coordination. Review relay configurations.")
    
    if metrics['total_tmt'] < -500:
        assessment['recommendations'].append("🔴 CRITICAL: Very negative total TMT. Multiple severe miscoordinations.")
    
    if metrics['good_scenarios_percentage'] < 20:
        assessment['recommendations'].append("🟡 WARNING: Few scenarios with acceptable coordination.")
    
    if metrics['critical_scenarios_count'] > metrics['total_scenarios'] * 0.3:
        assessment['recommendations'].append("🟡 WARNING: More than 30% of scenarios have critical TMT.")
    
    return assessment

def print_detailed_analysis(metrics, assessment):
    """
    Imprime un análisis detallado y profesional de los resultados.
    """
    print("\n" + "="*80)
    print("📊 ANÁLISIS DETALLADO DE COORDINACIÓN DE RELÉS")
    print("="*80)
    
    print(f"\n📈 MÉTRICAS GENERALES:")
    print(f"   • Total de escenarios analizados: {metrics['total_scenarios']}")
    print(f"   • Total de pares de relés: {metrics['total_pairs']:,}")
    print(f"   • Pares coordinados: {metrics['coordinated_pairs']:,} ({metrics['avg_coordination']:.2f}%)")
    print(f"   • Pares descoordinados: {metrics['uncoordinated_pairs']:,}")
    print(f"   • TMT total acumulado: {metrics['total_tmt']:.4f} segundos")
    
    print(f"\n📊 ESTADÍSTICAS DESCRIPTIVAS:")
    print(f"   • Coordinación promedio: {metrics['avg_coordination']:.2f}% ± {metrics['coordination_std']:.2f}%")
    print(f"   • Coordinación mediana: {metrics['coordination_median']:.2f}%")
    print(f"   • Rango de coordinación: {metrics['coordination_range']:.2f}%")
    print(f"   • TMT promedio: {metrics['tmt_median']:.4f} segundos")
    
    print(f"\n🎯 EVALUACIÓN DE CALIDAD:")
    print(f"   • Escenarios con coordinación ≥80%: {metrics['good_scenarios_count']} ({metrics['good_scenarios_percentage']:.1f}%)")
    print(f"   • Escenarios con TMT crítico: {metrics['critical_scenarios_count']} ({metrics['critical_scenarios_percentage']:.1f}%)")
    
    print(f"\n🏆 CALIFICACIÓN GENERAL: {assessment['overall_quality']}")
    print(f"   • Estado de coordinación: {assessment['coordination_status']}")
    print(f"   • Estado de TMT: {assessment['tmt_status']}")
    
    if assessment['recommendations']:
        print(f"\n💡 RECOMENDACIONES:")
        for i, rec in enumerate(assessment['recommendations'], 1):
            print(f"   {i}. {rec}")
    
    print("="*80)

print("✅ Funciones auxiliares avanzadas definidas.")


✅ Funciones auxiliares avanzadas definidas.


In [23]:
# %% Función Principal de Análisis Mejorada
def analyze_all_scenarios():
    """
    Carga datos de pares de relés, calcula TMT y coordinación por escenario,
    genera análisis estadístico avanzado y métricas de calidad.
    """
    try:
        # 1. Cargar el archivo JSON
        relay_pairs_data = load_relay_data(config.INPUT_FILE)

        # Diccionarios para almacenar datos por escenario
        scenario_results = defaultdict(lambda: {
            'tmt': 0.0,
            'coordinated': 0,
            'uncoordinated': 0,
            'total_valid': 0,
            'time_differences': [],  # Para análisis estadístico
            'miscoordination_times': []  # Para análisis de descoordinación
        })

        # 2. Procesar cada par y agrupar por escenario
        print("🔄 Calculando TMT y coordinación por escenario...")
        total_pairs_read = 0
        skipped_pairs_count = 0

        for pair_entry in relay_pairs_data:
            total_pairs_read += 1
            
            # Validar estructura de entrada
            if not isinstance(pair_entry, dict):
                print(f"⚠️  Warning: Entrada omitida no es un diccionario: {pair_entry}")
                skipped_pairs_count += 1
                continue

            scenario_id = pair_entry.get("scenario_id")
            if not scenario_id:
                print(f"⚠️  Warning: Entrada omitida por falta de 'scenario_id': {pair_entry.get('pair_id', 'ID desconocido')}")
                skipped_pairs_count += 1
                continue

            main_relay_info = pair_entry.get('main_relay')
            backup_relay_info = pair_entry.get('backup_relay')

            if not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
                print(f"⚠️  Warning: Datos de relé inválidos para par en escenario '{scenario_id}'. Par ID: {pair_entry.get('pair_id', 'ID desconocido')}")
                skipped_pairs_count += 1
                continue

            main_time = main_relay_info.get('Time_out')
            backup_time = backup_relay_info.get('Time_out')

            if not isinstance(main_time, (int, float)) or not isinstance(backup_time, (int, float)):
                print(f"⚠️  Warning: Tiempos inválidos ('{main_time}', '{backup_time}') para par en escenario '{scenario_id}'. Par ID: {pair_entry.get('pair_id', 'ID desconocido')}")
                skipped_pairs_count += 1
                continue

            if main_time < 0 or backup_time < 0:
                print(f"⚠️  Warning: Tiempos negativos ('{main_time}', '{backup_time}') en par '{pair_entry.get('pair_id', 'ID desconocido')}' (Esc: '{scenario_id}'). Omitiendo par.")
                skipped_pairs_count += 1
                continue

            # Calcular TMT y coordinación
            delta_t = backup_time - main_time - config.CTI
            mt = (delta_t - abs(delta_t)) / 2  # Penalización solo si delta_t < 0

            # Almacenar datos para análisis estadístico
            scenario_results[scenario_id]['time_differences'].append(delta_t)
            if delta_t < 0:
                scenario_results[scenario_id]['miscoordination_times'].append(abs(delta_t))

            scenario_results[scenario_id]['tmt'] += mt
            scenario_results[scenario_id]['total_valid'] += 1

            if delta_t >= 0:
                scenario_results[scenario_id]['coordinated'] += 1
            else:
                scenario_results[scenario_id]['uncoordinated'] += 1

        if not scenario_results:
            print("\n❌ Error CRÍTICO: No se encontraron datos de escenarios válidos para procesar.")
            return None, None, None

        # 3. Preparar datos para graficar y analizar
        print("📊 Preparando datos para visualización y análisis estadístico...")
        data_for_analysis = []
        
        for scenario, data in scenario_results.items():
            total_valid_pairs = data['total_valid']
            percentage_coordination = (data['coordinated'] / total_valid_pairs * 100) if total_valid_pairs > 0 else 0
            
            # Calcular métricas adicionales por escenario
            avg_time_diff = np.mean(data['time_differences']) if data['time_differences'] else 0
            std_time_diff = np.std(data['time_differences']) if data['time_differences'] else 0
            max_miscoordination = max(data['miscoordination_times']) if data['miscoordination_times'] else 0
            avg_miscoordination = np.mean(data['miscoordination_times']) if data['miscoordination_times'] else 0
            
            # Clasificar calidad del escenario
            if percentage_coordination >= config.MIN_COORDINATION_THRESHOLD:
                quality = 'GOOD'
            elif percentage_coordination >= 60:
                quality = 'REGULAR'
            else:
                quality = 'CRITICAL'

            data_for_analysis.append({
                'Escenario': scenario,
                'TMT': data['tmt'],
                'Pares Coordinados': data['coordinated'],
                'Pares Descoordinados': data['uncoordinated'],
                'Total Pares Válidos': total_valid_pairs,
                'Porcentaje Coordinación': percentage_coordination,
                'Tiempo Diferencia Promedio': avg_time_diff,
                'Desviación Estándar Tiempo': std_time_diff,
                'Máxima Descoordinación': max_miscoordination,
                'Descoordinación Promedio': avg_miscoordination,
                'Calidad': quality
            })

        df = pd.DataFrame(data_for_analysis)

        # --- ORDENAMIENTO NUMÉRICO ---
        df['Scenario_Num'] = df['Escenario'].apply(extract_scenario_number)
        df = df.sort_values(by='Scenario_Num').reset_index(drop=True)
        print("✅ Datos ordenados por escenario numéricamente.")

        # --- IMPRESIÓN DE TABLA DETALLADA MEJORADA ---
        print("\n" + "="*120)
        print("📋 RESULTADOS DETALLADOS POR ESCENARIO (ORDENADOS)")
        print("="*120)
        print(f"{'Escenario':<20} | {'Pares':>6} | {'Coord.':>6} | {'Desc.':>6} | {'% Coord.':>8} | {'TMT':>10} | {'Calidad':>8} | {'Max Desc.':>10}")
        print("-" * 120)
        for index, row in df.iterrows():
            quality_icon = "🟢" if row['Calidad'] == 'GOOD' else "🟡" if row['Calidad'] == 'REGULAR' else "🔴"
            print(f"{row['Escenario']:<20} | {int(row['Total Pares Válidos']):>6} | {int(row['Pares Coordinados']):>6} | {int(row['Pares Descoordinados']):>6} | {row['Porcentaje Coordinación']:>7.1f}% | {row['TMT']:>9.4f} | {quality_icon}{row['Calidad']:<7} | {row['Máxima Descoordinación']:>9.3f}")
        print("-" * 120)
        print(f"📊 Total pares leídos: {total_pairs_read:,}, Omitidos: {skipped_pairs_count}, Válidos Procesados: {df['Total Pares Válidos'].sum():,}")

        # --- ANÁLISIS ESTADÍSTICO AVANZADO ---
        metrics = calculate_advanced_metrics(df)
        assessment = generate_quality_assessment(metrics)
        print_detailed_analysis(metrics, assessment)

        return df, scenario_results, metrics

    except Exception as e:
        print(f"❌ Error durante el análisis: {e}")
        traceback.print_exc()
        return None, None, None

print("✅ Función de análisis mejorada definida.")


✅ Función de análisis mejorada definida.


In [24]:
# %% Ejecutar Análisis Completo
print("🚀 Iniciando análisis completo de coordinación de relés...")
print(f"⏰ Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Ejecutar análisis principal
df_results, scenario_data, metrics = analyze_all_scenarios()

if df_results is not None and not df_results.empty:
    print(f"\n✅ Análisis completado exitosamente.")
    print(f"📊 Se procesaron {len(df_results)} escenarios.")
    print(f"🔢 TMT total: {df_results['TMT'].sum():.4f}")
    print(f"📈 Coordinación promedio: {df_results['Porcentaje Coordinación'].mean():.2f}%")
    
    print("\n📊 Los datos están listos para visualización.")
    print("   Ejecuta la siguiente celda para generar las visualizaciones.")
    
    # Mostrar resumen rápido de los resultados más críticos
    print("\n🔍 RESUMEN DE RESULTADOS CRÍTICOS:")
    worst_scenarios = df_results.nsmallest(3, 'TMT')
    print("   Top 3 escenarios con mayor penalización TMT:")
    for _, row in worst_scenarios.iterrows():
        print(f"   • {row['Escenario']}: TMT = {row['TMT']:.4f}, Coordinación = {row['Porcentaje Coordinación']:.1f}%")
    
    critical_scenarios = df_results[df_results['Porcentaje Coordinación'] < 20]
    print(f"\n   Escenarios con coordinación < 20%: {len(critical_scenarios)} de {len(df_results)}")
    
    if len(critical_scenarios) > 0:
        print("   Peores casos:")
        for _, row in critical_scenarios.head(3).iterrows():
            print(f"   • {row['Escenario']}: {row['Porcentaje Coordinación']:.1f}% coordinación")
    
else:
    print("❌ Error: No se pudieron generar resultados del análisis.")


🚀 Iniciando análisis completo de coordinación de relés...
⏰ Timestamp: 2025-10-06 19:08:56
📊 Loading data from: /Users/gustavo/Documents/Projects/TESIS_UNAL/AutoDOC-MG/data/raw/automation_results.json
✅ Data loaded successfully. Total entries: 6,800
🔄 Calculando TMT y coordinación por escenario...
📊 Preparando datos para visualización y análisis estadístico...
✅ Datos ordenados por escenario numéricamente.

📋 RESULTADOS DETALLADOS POR ESCENARIO (ORDENADOS)
Escenario            |  Pares | Coord. |  Desc. | % Coord. |        TMT |  Calidad |  Max Desc.
------------------------------------------------------------------------------------------------------------------------
scenario_1           |    100 |     12 |     88 |    12.0% |  -15.8543 | 🔴CRITICAL |     0.384
scenario_2           |    100 |     12 |     88 |    12.0% |  -18.9624 | 🔴CRITICAL |     0.982
scenario_3           |    100 |     13 |     87 |    13.0% |  -17.8215 | 🔴CRITICAL |     0.511
scenario_4           |    100 |     1

In [25]:
# %% Funciones de Visualización Esenciales
def create_essential_visualizations(df, config):
    """
    Crea solo las visualizaciones más importantes para el análisis.
    """
    if df is None or df.empty:
        print("❌ No hay datos para visualizar.")
        return []
    
    figures = []
    
    # 1. Gráfico de Coordinación con Código de Colores por Calidad
    fig_coord = create_coordination_quality_chart(df, config)
    figures.append(('coordination_quality', fig_coord))
    
    # 2. Gráfico de TMT con Análisis de Severidad
    fig_tmt = create_tmt_severity_chart(df, config)
    figures.append(('tmt_severity', fig_tmt))
    
    # 3. Dashboard de Métricas Clave
    fig_dashboard = create_metrics_dashboard(df, config)
    figures.append(('metrics_dashboard', fig_dashboard))
    
    return figures

def create_coordination_quality_chart(df, config):
    """Gráfico de coordinación con código de colores por calidad"""
    fig = px.bar(
        df,
        x='Escenario',
        y='Porcentaje Coordinación',
        color='Calidad',
        color_discrete_map={
            'GOOD': config.COLORS['good'],
            'REGULAR': config.COLORS['warning'],
            'CRITICAL': config.COLORS['critical']
        },
        text='Porcentaje Coordinación',
        title='Coordination Quality Analysis by Scenario',
        labels={'Porcentaje Coordinación': 'Coordination (%)', 'Escenario': 'Operational Scenario'},
        hover_data=['Pares Coordinados', 'Pares Descoordinados', 'TMT']
    )
    
    fig.update_traces(
        texttemplate='%{text:.1f}%',
        textposition='outside',
        textfont_size=config.TEXT_ON_BAR_FONT_SIZE
    )
    
    fig.update_layout(
        template=config.PLOT_TEMPLATE,
        width=config.FIG_WIDTH,
        height=config.FIG_HEIGHT,
        title_font_size=config.TITLE_FONT_SIZE,
        xaxis_title='<b>Operational Scenario</b>',
        yaxis_title='<b>Coordination (%)</b>',
        xaxis={'categoryorder': 'array', 'categoryarray': df['Escenario'].tolist()},
        yaxis=dict(range=[0, 105]),
        margin=dict(l=80, r=50, t=100, b=80)
    )
    
    return fig

def create_tmt_severity_chart(df, config):
    """Gráfico de TMT con análisis de severidad"""
    # Clasificar TMT por severidad
    df['Severidad'] = df['TMT'].apply(lambda x: 
        'CRITICAL' if x <= config.CRITICAL_TMT_THRESHOLD else
        'HIGH' if x <= -10 else
        'MODERATE' if x <= -5 else
        'LOW' if x < 0 else 'NONE'
    )
    
    fig = px.bar(
        df,
        x='Escenario',
        y='TMT',
        color='Severidad',
        color_discrete_map={
            'CRITICAL': config.COLORS['critical'],
            'HIGH': config.COLORS['uncoordinated'],
            'MODERATE': config.COLORS['warning'],
            'LOW': '#FFA500',
            'NONE': config.COLORS['good']
        },
        text='TMT',
        title='Análisis de Severidad de TMT por Escenario',
        labels={'TMT': 'TMT Acumulado (s)', 'Escenario': 'Operational Scenario'},
        hover_data=['Porcentaje Coordinación', 'Pares Coordinados', 'Pares Descoordinados']
    )
    
    fig.update_traces(
        texttemplate='%{text:.3f}',
        textposition='outside',
        textfont_size=config.TEXT_ON_BAR_FONT_SIZE
    )
    
    fig.update_layout(
        template=config.PLOT_TEMPLATE,
        width=config.FIG_WIDTH,
        height=config.FIG_HEIGHT,
        title_font_size=config.TITLE_FONT_SIZE,
        xaxis_title='<b>Operational Scenario</b>',
        yaxis_title='<b>TMT Acumulado (s)</b>',
        xaxis={'categoryorder': 'array', 'categoryarray': df['Escenario'].tolist()},
        margin=dict(l=80, r=50, t=100, b=80)
    )
    
    return fig

def create_metrics_dashboard(df, config):
    """Dashboard con métricas clave"""
    # Crear subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Distribución de Coordinación',
            'Distribución de TMT',
            'Coordinación vs TMT',
            'Resumen de Calidad'
        ),
        specs=[[{"type": "histogram"}, {"type": "histogram"}],
               [{"type": "scatter"}, {"type": "bar"}]]
    )
    
    # Histograma de coordinación
    fig.add_trace(
        go.Histogram(x=df['Porcentaje Coordinación'], nbinsx=15, name='Coordinación', 
                    marker_color=config.COLORS['primary']),
        row=1, col=1
    )
    
    # Histograma de TMT
    fig.add_trace(
        go.Histogram(x=df['TMT'], nbinsx=15, name='TMT', 
                    marker_color=config.COLORS['secondary']),
        row=1, col=2
    )
    
    # Scatter plot: Coordinación vs TMT
    fig.add_trace(
        go.Scatter(x=df['Porcentaje Coordinación'], y=df['TMT'], mode='markers',
                  marker=dict(color=df['Porcentaje Coordinación'], colorscale='RdYlGn',
                             size=6, showscale=True),
                  text=df['Escenario'], name='Escenarios'),
        row=2, col=1
    )
    
    # Resumen de calidad
    quality_counts = df['Calidad'].value_counts()
    colors_list = [config.COLORS['good'], config.COLORS['warning'], config.COLORS['critical']]
    fig.add_trace(
        go.Bar(x=quality_counts.index, y=quality_counts.values,
               marker_color=colors_list[:len(quality_counts)],
               name='Calidad'),
        row=2, col=2
    )
    
    fig.update_layout(
        template=config.PLOT_TEMPLATE,
        width=config.FIG_WIDTH + 200,
        height=config.FIG_HEIGHT + 200,
        title_text="Dashboard de Análisis de Coordinación de Relés",
        title_font_size=config.TITLE_FONT_SIZE,
        showlegend=False
    )
    
    return fig

print("✅ Funciones de visualización esenciales definidas.")


✅ Funciones de visualización esenciales definidas.


In [26]:
# %% Generar y Mostrar Visualizaciones Esenciales
if df_results is not None and not df_results.empty:
    print("🎨 Generando visualizaciones esenciales...")
    
    # Generar solo las visualizaciones más importantes
    figures = create_essential_visualizations(df_results, config)
    
    print(f"✅ Se generaron {len(figures)} visualizaciones esenciales.")
    
    # Mostrar todas las visualizaciones esenciales
    print("\n📊 Mostrando visualizaciones:")
    for name, fig in figures:
        if fig is not None:
            print(f"   • Showing: {name}")
            fig.show()
else:
    print("❌ No hay datos para generar visualizaciones. Ejecuta primero la celda de análisis.")


🎨 Generando visualizaciones esenciales...
✅ Se generaron 3 visualizaciones esenciales.

📊 Mostrando visualizaciones:
   • Showing: coordination_quality


   • Showing: tmt_severity


   • Showing: metrics_dashboard


In [27]:
# %% Funciones de Generación de Reportes Detallados
def generate_comprehensive_report(df, metrics, figures, config):
    """
    Genera un reporte completo y detallado del análisis de coordinación de relés.
    Incluye análisis estadístico, visualizaciones y recomendaciones.
    """
    if df is None or df.empty:
        print("❌ No hay datos para generar el reporte.")
        return None
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    print(f"\n📝 Generando reporte detallado con timestamp: {timestamp}")
    
    # Crear directorio de reportes si no existe
    reports_dir = config.RESULTS_DIR / "reports"
    reports_dir.mkdir(parents=True, exist_ok=True)
    
    # Nombre del notebook para identificación
    notebook_name = "tmt_analysis_all_scenarios"
    
    # 1. Reporte de texto detallado
    text_report_path = reports_dir / f"{notebook_name}_detailed_report_{timestamp}.txt"
    generate_detailed_text_report(df, metrics, text_report_path, config)
    
    # 2. Reporte HTML interactivo - DISABLED
    # html_report_path = reports_dir / f"{notebook_name}_interactive_report_{timestamp}.html"
    # generate_html_report(df, metrics, figures, html_report_path, config)
    
    # 3. Reporte ejecutivo (resumen)
    executive_report_path = reports_dir / f"{notebook_name}_executive_summary_{timestamp}.txt"
    generate_executive_summary(df, metrics, executive_report_path, config)
    
    # 4. Guardar datos en CSV
    csv_path = config.TABLES_DIR / f"{notebook_name}_results_{timestamp}.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8')
    
    # 5. Guardar métricas en JSON
    metrics_path = config.TABLES_DIR / f"{notebook_name}_metrics_{timestamp}.json"
    save_metrics_json(metrics, metrics_path)
    
    # 6. Guardar visualizaciones
    save_visualizations(figures, config, timestamp)
    
    return {
        'text_report': text_report_path,
        'executive_summary': executive_report_path,
        'csv_data': csv_path,
        'metrics_json': metrics_path,
        'timestamp': timestamp
    }

def generate_detailed_text_report(df, metrics, report_path, config):
    """Generates a detailed and professional text report"""
    
    with open(report_path, 'w', encoding='utf-8') as f:
        # Report header
        f.write("="*100 + "\n")
        f.write("DETAILED RELAY COORDINATION ANALYSIS REPORT\n")
        f.write("="*100 + "\n")
        f.write(f"Generation date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Project: {config.PROJECT_ROOT.name}\n")
        f.write(f"Analyzed file: {config.INPUT_FILE.name}\n")
        f.write(f"CTI configuration: {config.CTI} seconds\n")
        f.write(f"Coordination threshold: {config.MIN_COORDINATION_THRESHOLD}%\n")
        f.write(f"Critical TMT threshold: {config.CRITICAL_TMT_THRESHOLD}\n\n")
        
        # Executive summary
        f.write("EXECUTIVE SUMMARY\n")
        f.write("-"*50 + "\n")
        f.write(f"• Total scenarios analyzed: {metrics['total_scenarios']}\n")
        f.write(f"• Total relay pairs: {metrics['total_pairs']:,}\n")
        f.write(f"• Coordinated pairs: {metrics['coordinated_pairs']:,} ({metrics['avg_coordination']:.2f}%)\n")
        f.write(f"• Uncoordinated pairs: {metrics['uncoordinated_pairs']:,}\n")
        f.write(f"• Total accumulated TMT: {metrics['total_tmt']:.4f} seconds\n")
        f.write(f"• Scenarios with coordination ≥{config.MIN_COORDINATION_THRESHOLD}%: {metrics['good_scenarios_count']} ({metrics['good_scenarios_percentage']:.1f}%)\n")
        f.write(f"• Critical scenarios (TMT ≤ {config.CRITICAL_TMT_THRESHOLD}): {metrics['critical_scenarios_count']} ({metrics['critical_scenarios_percentage']:.1f}%)\n\n")
        
        # Detailed statistical analysis
        f.write("DETAILED STATISTICAL ANALYSIS\n")
        f.write("-"*50 + "\n")
        f.write(f"Coordination:\n")
        f.write(f"  • Average: {metrics['avg_coordination']:.2f}% ± {metrics['coordination_std']:.2f}%\n")
        f.write(f"  • Median: {metrics['coordination_median']:.2f}%\n")
        f.write(f"  • Range: {metrics['coordination_range']:.2f}%\n")
        f.write(f"  • Minimum: {df['Porcentaje Coordinación'].min():.2f}%\n")
        f.write(f"  • Maximum: {df['Porcentaje Coordinación'].max():.2f}%\n\n")
        
        f.write(f"TMT (Total Miscoordination Time):\n")
        f.write(f"  • Average: {metrics['tmt_median']:.4f} seconds\n")
        f.write(f"  • Standard deviation: {metrics['tmt_std']:.4f} seconds\n")
        f.write(f"  • Range: {metrics['tmt_range']:.4f} seconds\n")
        f.write(f"  • Minimum: {df['TMT'].min():.4f} seconds\n")
        f.write(f"  • Maximum: {df['TMT'].max():.4f} seconds\n\n")
        
        # Analysis by scenario
        f.write("DETAILED ANALYSIS BY SCENARIO\n")
        f.write("-"*50 + "\n")
        f.write(f"{'Scenario':<25} | {'Pairs':>6} | {'Coord.':>6} | {'Uncoord.':>8} | {'% Coord.':>8} | {'TMT':>10} | {'Quality':>8} | {'Max Uncoord.':>12}\n")
        f.write("-" * 100 + "\n")
        
        for _, row in df.iterrows():
            quality_icon = "🟢" if row['Calidad'] == 'GOOD' else "🟡" if row['Calidad'] == 'REGULAR' else "🔴"
            f.write(f"{row['Escenario']:<25} | {int(row['Total Pares Válidos']):>6} | {int(row['Pares Coordinados']):>6} | {int(row['Pares Descoordinados']):>8} | {row['Porcentaje Coordinación']:>7.1f}% | {row['TMT']:>9.4f} | {quality_icon}{row['Calidad']:<7} | {row['Máxima Descoordinación']:>11.3f}\n")
        
        f.write("-" * 100 + "\n\n")
        
        # Problematic scenarios identification
        f.write("PROBLEMATIC SCENARIOS IDENTIFICATION\n")
        f.write("-"*50 + "\n")
        
        # Worst scenarios by TMT
        worst_tmt = df.nsmallest(5, 'TMT')
        f.write("Top 5 scenarios with highest TMT penalty:\n")
        for i, (_, row) in enumerate(worst_tmt.iterrows(), 1):
            f.write(f"  {i}. {row['Escenario']}: TMT = {row['TMT']:.4f}s, Coordination = {row['Porcentaje Coordinación']:.1f}%\n")
        
        f.write("\n")
        
        # Scenarios with low coordination
        low_coordination = df[df['Porcentaje Coordinación'] < 50]
        if len(low_coordination) > 0:
            f.write(f"Scenarios with coordination < 50% ({len(low_coordination)} scenarios):\n")
            for _, row in low_coordination.iterrows():
                f.write(f"  • {row['Escenario']}: {row['Porcentaje Coordinación']:.1f}% coordination, TMT = {row['TMT']:.4f}s\n")
        else:
            f.write("✅ No scenarios with coordination < 50%\n")
        
        f.write("\n")
        
        # Recommendations
        f.write("TECHNICAL RECOMMENDATIONS\n")
        f.write("-"*50 + "\n")
        
        assessment = generate_quality_assessment(metrics)
        for i, rec in enumerate(assessment['recommendations'], 1):
            f.write(f"{i}. {rec}\n")
        
        # Additional recommendations based on analysis
        if metrics['avg_coordination'] < 70:
            f.write(f"{len(assessment['recommendations']) + 1}. 🔴 URGENT: Average coordination ({metrics['avg_coordination']:.1f}%) is well below recommended threshold. Review relay configurations.\n")
        
        if metrics['critical_scenarios_percentage'] > 30:
            f.write(f"{len(assessment['recommendations']) + 2}. 🟡 ATTENTION: More than 30% of scenarios have critical TMT. Consider coordination time adjustments.\n")
        
        if metrics['coordination_std'] > 30:
            f.write(f"{len(assessment['recommendations']) + 3}. 🟡 VARIABILITY: High coordination variability (σ = {metrics['coordination_std']:.1f}%). Review configuration consistency.\n")
        
        f.write("\n")
        
        # Technical information
        f.write("TECHNICAL INFORMATION\n")
        f.write("-"*50 + "\n")
        f.write(f"• CTI (Coordination Time Interval): {config.CTI} seconds\n")
        f.write(f"• Acceptable coordination threshold: {config.MIN_COORDINATION_THRESHOLD}%\n")
        f.write(f"• Critical TMT threshold: {config.CRITICAL_TMT_THRESHOLD}\n")
        f.write(f"• Total processed pairs: {metrics['total_pairs']:,}\n")
        f.write(f"• Coordination success rate: {metrics['avg_coordination']:.2f}%\n")
        f.write(f"• System efficiency: {100 - metrics['critical_scenarios_percentage']:.1f}%\n\n")
        
        f.write("="*100 + "\n")
        f.write("END OF REPORT\n")
        f.write("="*100 + "\n")
    
    print(f"✅ Detailed report saved in: {report_path}")

def generate_html_report(df, metrics, figures, html_path, config):
    """Genera un reporte HTML interactivo con visualizaciones"""
    
    html_content = f"""
    <!DOCTYPE html>
    <html lang="es">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Reporte de Análisis TMT - {datetime.now().strftime('%Y-%m-%d')}</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; background-color: #f5f5f5; }}
            .container {{ max-width: 1200px; margin: 0 auto; background: white; padding: 20px; border-radius: 10px; box-shadow: 0 0 10px rgba(0,0,0,0.1); }}
            .header {{ text-align: center; border-bottom: 3px solid #4169E1; padding-bottom: 20px; margin-bottom: 30px; }}
            .section {{ margin: 30px 0; }}
            .metric-card {{ background: #f8f9fa; padding: 15px; border-radius: 8px; margin: 10px 0; border-left: 4px solid #4169E1; }}
            .critical {{ border-left-color: #DC143C; }}
            .warning {{ border-left-color: #FFD700; }}
            .good {{ border-left-color: #32CD32; }}
            table {{ width: 100%; border-collapse: collapse; margin: 20px 0; }}
            th, td {{ padding: 12px; text-align: left; border-bottom: 1px solid #ddd; }}
            th {{ background-color: #4169E1; color: white; }}
            .plot-container {{ margin: 30px 0; text-align: center; }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h1>📊 Reporte de Análisis de Coordinación de Relés</h1>
                <p>Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
                <p>Proyecto: {config.PROJECT_ROOT.name}</p>
            </div>
            
            <div class="section">
                <h2>📈 Resumen Ejecutivo</h2>
                <div class="metric-card">
                    <h3>Métricas Generales</h3>
                    <p><strong>Total de escenarios:</strong> {metrics['total_scenarios']}</p>
                    <p><strong>Total de pares de relés:</strong> {metrics['total_pairs']:,}</p>
                    <p><strong>Coordinación promedio:</strong> {metrics['avg_coordination']:.2f}%</p>
                    <p><strong>TMT total acumulado:</strong> {metrics['total_tmt']:.4f} segundos</p>
                </div>
                
                <div class="metric-card {'critical' if metrics['avg_coordination'] < 70 else 'warning' if metrics['avg_coordination'] < 80 else 'good'}">
                    <h3>Estado de Calidad</h3>
                    <p><strong>Escenarios con coordinación ≥{config.MIN_COORDINATION_THRESHOLD}%:</strong> {metrics['good_scenarios_count']} ({metrics['good_scenarios_percentage']:.1f}%)</p>
                    <p><strong>Escenarios críticos:</strong> {metrics['critical_scenarios_count']} ({metrics['critical_scenarios_percentage']:.1f}%)</p>
                </div>
            </div>
            
            <div class="section">
                <h2>📊 Análisis por Escenario</h2>
                <table>
                    <thead>
                        <tr>
                            <th>Escenario</th>
                            <th>Pares</th>
                            <th>Coordinados</th>
                            <th>Descoordinados</th>
                            <th>% Coordinación</th>
                            <th>TMT</th>
                            <th>Calidad</th>
                        </tr>
                    </thead>
                    <tbody>
    """
    
    for _, row in df.iterrows():
        quality_icon = "🟢" if row['Calidad'] == 'GOOD' else "🟡" if row['Calidad'] == 'REGULAR' else "🔴"
        html_content += f"""
                        <tr>
                            <td>{row['Escenario']}</td>
                            <td>{int(row['Total Pares Válidos'])}</td>
                            <td>{int(row['Pares Coordinados'])}</td>
                            <td>{int(row['Pares Descoordinados'])}</td>
                            <td>{row['Porcentaje Coordinación']:.1f}%</td>
                            <td>{row['TMT']:.4f}</td>
                            <td>{quality_icon} {row['Calidad']}</td>
                        </tr>
        """
    
    html_content += """
                    </tbody>
                </table>
            </div>
            
            <div class="section">
                <h2>📈 Visualizaciones</h2>
                <p><em>Las visualizaciones interactivas se generan automáticamente y se guardan como archivos HTML separados.</em></p>
            </div>
            
            <div class="section">
                <h2>💡 Recomendaciones</h2>
    """
    
    assessment = generate_quality_assessment(metrics)
    for i, rec in enumerate(assessment['recommendations'], 1):
        html_content += f"<p>{i}. {rec}</p>"
    
    html_content += """
            </div>
        </div>
    </body>
    </html>
    """
    
    with open(html_path, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"✅ Reporte HTML guardado en: {html_path}")

def generate_executive_summary(df, metrics, summary_path, config):
    """Generates a concise executive summary"""
    
    with open(summary_path, 'w', encoding='utf-8') as f:
        f.write("EXECUTIVE SUMMARY - RELAY COORDINATION ANALYSIS\n")
        f.write("="*60 + "\n\n")
        
        f.write(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"📁 Project: {config.PROJECT_ROOT.name}\n")
        f.write(f"📊 File: {config.INPUT_FILE.name}\n\n")
        
        f.write("KEY RESULTS:\n")
        f.write("-" * 30 + "\n")
        f.write(f"• Scenarios analyzed: {metrics['total_scenarios']}\n")
        f.write(f"• Average coordination: {metrics['avg_coordination']:.1f}%\n")
        f.write(f"• Successful scenarios (≥{config.MIN_COORDINATION_THRESHOLD}%): {metrics['good_scenarios_percentage']:.1f}%\n")
        f.write(f"• Critical scenarios: {metrics['critical_scenarios_percentage']:.1f}%\n")
        f.write(f"• Total TMT: {metrics['total_tmt']:.2f} seconds\n\n")
        
        f.write("GENERAL STATUS:\n")
        f.write("-" * 30 + "\n")
        if metrics['avg_coordination'] >= 80:
            f.write("✅ SYSTEM IN GOOD CONDITION\n")
        elif metrics['avg_coordination'] >= 60:
            f.write("⚠️  SYSTEM REQUIRES ATTENTION\n")
        else:
            f.write("🔴 CRITICAL SYSTEM - IMMEDIATE ACTION REQUIRED\n")
        
        f.write("\nRECOMMENDED ACTION:\n")
        f.write("-" * 30 + "\n")
        if metrics['avg_coordination'] < 70:
            f.write("• Review relay configurations urgently\n")
        if metrics['critical_scenarios_percentage'] > 20:
            f.write("• Adjust coordination times\n")
        if metrics['coordination_std'] > 25:
            f.write("• Standardize configurations\n")
        
        f.write("\n" + "="*60 + "\n")
    
    print(f"✅ Executive summary saved in: {summary_path}")

def save_metrics_json(metrics, metrics_path):
    """Guarda las métricas en formato JSON"""
    metrics_serializable = {}
    for key, value in metrics.items():
        if hasattr(value, 'item'):  # numpy types
            metrics_serializable[key] = value.item()
        else:
            metrics_serializable[key] = value
    
    with open(metrics_path, 'w', encoding='utf-8') as f:
        json.dump(metrics_serializable, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Metrics saved in: {metrics_path}")

def save_visualizations(figures, config, timestamp):
    """Saves visualizations in PNG format with descriptive names"""
    saved_plots = []
    notebook_name = "tmt_analysis_all_scenarios"  # Notebook name for identification
    
    for name, fig in figures:
        if fig is not None:
            # High resolution PNG with notebook name
            try:
                png_path = config.OUTPUT_DIR / f"{notebook_name}_{name}_{timestamp}.png"
                fig.write_image(png_path, scale=2, width=config.FIG_WIDTH, height=config.FIG_HEIGHT)
                saved_plots.append(png_path)
                print(f"✅ PNG saved: {png_path.name}")
            except Exception as e:
                print(f"⚠️  Could not save PNG for {name}: {e}")
                # Try with kaleido if available
                try:
                    import kaleido
                    fig.write_image(png_path, scale=2, width=config.FIG_WIDTH, height=config.FIG_HEIGHT)
                    saved_plots.append(png_path)
                    print(f"✅ PNG saved (with kaleido): {png_path.name}")
                except ImportError:
                    print(f"💡 Install kaleido to export PNG: pip install kaleido")
                except Exception as e2:
                    print(f"❌ Error saving PNG: {e2}")
    
    print(f"✅ {len(saved_plots)} visualizations saved in: {config.OUTPUT_DIR}")
    print(f"📁 PNG files generated:")
    png_files = [f for f in saved_plots if f.suffix == '.png']
    for png_file in png_files:
        print(f"   • {png_file.name}")

def generate_quality_assessment(metrics):
    """Generates a quality assessment based on calculated metrics"""
    assessment = {
        'overall_quality': 'CRITICAL',
        'coordination_status': 'INSUFFICIENT',
        'tmt_status': 'CRITICAL',
        'recommendations': []
    }
    
    # Coordination evaluation
    if metrics['avg_coordination'] >= 90:
        assessment['coordination_status'] = 'EXCELLENT'
    elif metrics['avg_coordination'] >= 80:
        assessment['coordination_status'] = 'GOOD'
    elif metrics['avg_coordination'] >= 60:
        assessment['coordination_status'] = 'REGULAR'
    else:
        assessment['coordination_status'] = 'INSUFFICIENT'
    
    # TMT evaluation
    if metrics['total_tmt'] >= -100:
        assessment['tmt_status'] = 'GOOD'
    elif metrics['total_tmt'] >= -500:
        assessment['tmt_status'] = 'REGULAR'
    else:
        assessment['tmt_status'] = 'CRITICAL'
    
    # Overall evaluation
    if assessment['coordination_status'] in ['EXCELLENT', 'GOOD'] and assessment['tmt_status'] in ['GOOD', 'REGULAR']:
        assessment['overall_quality'] = 'GOOD'
    elif assessment['coordination_status'] in ['REGULAR'] or assessment['tmt_status'] in ['REGULAR']:
        assessment['overall_quality'] = 'REGULAR'
    else:
        assessment['overall_quality'] = 'CRITICAL'
    
    # Generate recommendations
    if metrics['avg_coordination'] < 80:
        assessment['recommendations'].append("🔴 CRITICAL: Very low average coordination. Review relay configurations.")
    
    if metrics['total_tmt'] < -500:
        assessment['recommendations'].append("🔴 CRITICAL: Very negative total TMT. Multiple severe miscoordinations.")
    
    if metrics['good_scenarios_percentage'] < 20:
        assessment['recommendations'].append("🟡 WARNING: Few scenarios with acceptable coordination.")
    
    if metrics['critical_scenarios_count'] > metrics['total_scenarios'] * 0.3:
        assessment['recommendations'].append("🟡 WARNING: More than 30% of scenarios have critical TMT.")
    
    return assessment

print("✅ Funciones de generación de reportes definidas.")


✅ Funciones de generación de reportes definidas.


In [28]:
# %% Generar Reportes Detallados
if df_results is not None and not df_results.empty and 'figures' in locals():
    print("📝 Iniciando generación de reportes detallados...")
    
    # Generar reporte completo
    report_files = generate_comprehensive_report(df_results, metrics, figures, config)
    
    if report_files:
        print("\n🎉 REPORTES GENERADOS EXITOSAMENTE")
        print("="*60)
        print("📁 Archivos generados:")
        print(f"   • Reporte detallado: {report_files['text_report'].name}")
        print(f"   • Reporte HTML: {report_files['html_report'].name}")
        print(f"   • Resumen ejecutivo: {report_files['executive_summary'].name}")
        print(f"   • Datos CSV: {report_files['csv_data'].name}")
        print(f"   • Métricas JSON: {report_files['metrics_json'].name}")
        print(f"   • Timestamp: {report_files['timestamp']}")
        
        print("\n📂 Ubicación de archivos:")
        print(f"   • Reportes: {report_files['text_report'].parent}")
        print(f"   • Datos: {report_files['csv_data'].parent}")
        print(f"   • Visualizaciones: {config.OUTPUT_DIR}")
        
        print("\n🌍 Listo para compartir internacionalmente!")
        print("="*60)
    else:
        print("❌ Error al generar los reportes.")
else:
    print("❌ No hay datos para generar reportes.")
    print("   Asegúrate de ejecutar primero las celdas de análisis y visualización.")


📝 Iniciando generación de reportes detallados...

📝 Generando reporte detallado con timestamp: 20251006_190856
✅ Detailed report saved in: /Users/gustavo/Documents/Projects/TESIS_UNAL/AutoDOC-MG/results/reports/tmt_analysis_all_scenarios_detailed_report_20251006_190856.txt
✅ Executive summary saved in: /Users/gustavo/Documents/Projects/TESIS_UNAL/AutoDOC-MG/results/reports/tmt_analysis_all_scenarios_executive_summary_20251006_190856.txt
✅ Metrics saved in: /Users/gustavo/Documents/Projects/TESIS_UNAL/AutoDOC-MG/results/tables/tmt_analysis_all_scenarios_metrics_20251006_190856.json
⚠️  Could not save PNG for coordination_quality: 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome


❌ Error saving PNG: 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's ins

KeyError: 'html_report'

In [ ]:
# %% Generar Solo Imágenes PNG
def generate_png_images_only(figures, config):
    """Genera solo las imágenes PNG de alta resolución"""
    if not figures:
        print("❌ No hay figuras para convertir a PNG.")
        return
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    notebook_name = "tmt_analysis_all_scenarios"
    
    print(f"🖼️  Generando imágenes PNG con timestamp: {timestamp}")
    
    saved_pngs = []
    
    for name, fig in figures:
        if fig is not None:
            try:
                png_path = config.OUTPUT_DIR / f"{notebook_name}_{name}_{timestamp}.png"
                fig.write_image(png_path, scale=2, width=config.FIG_WIDTH, height=config.FIG_HEIGHT)
                saved_pngs.append(png_path)
                print(f"✅ PNG guardado: {png_path.name}")
            except Exception as e:
                print(f"⚠️  Error al guardar {name}.png: {e}")
                # Intentar con kaleido
                try:
                    import kaleido
                    fig.write_image(png_path, scale=2, width=config.FIG_WIDTH, height=config.FIG_HEIGHT)
                    saved_pngs.append(png_path)
                    print(f"✅ PNG guardado (con kaleido): {png_path.name}")
                except ImportError:
                    print(f"💡 Para exportar PNG instala: pip install kaleido")
                except Exception as e2:
                    print(f"❌ Error definitivo: {e2}")
    
    if saved_pngs:
        print(f"\n🎉 {len(saved_pngs)} imágenes PNG generadas exitosamente:")
        for png_file in saved_pngs:
            print(f"   📁 {png_file.name}")
        print(f"\n📂 Ubicación: {config.OUTPUT_DIR}")
    else:
        print("❌ No se pudieron generar imágenes PNG.")

# Ejecutar generación de PNG si hay figuras disponibles
if 'figures' in locals() and figures:
    generate_png_images_only(figures, config)
else:
    print("❌ No hay figuras disponibles. Ejecuta primero las celdas de análisis y visualización.")
